In [2]:
from google.cloud import speech_v1p1beta1 as speech
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='rich-suprstate-311107-b521ee51d373.json'

client = speech.SpeechClient()
encoding=speech.enums.RecognitionConfig.AudioEncoding

input = "stt_success/audio.raw"

config = {
    "language_code": 'en-US',
    "encoding": encoding.LINEAR16,
    "sample_rate_hertz": 22050,
    "speech_contexts": []
}

with open(input, "rb") as f:
    audio = {"content": f.read()}
    
response = client.recognize(config, audio)

for result in response.results: 
    print('Transcript: {}'.format(result.alternatives[0].transcript))

Transcript: how old is the Brooklyn Bridge


## Cantonese samples

In [3]:
input = "stt_success/MPF check restaurant surroundings.wav"

config = {
    "language_code": 'yue-Hant-HK',
    "encoding": encoding.LINEAR16,
    "sample_rate_hertz": 44100,
    "audio_channel_count":1,
#     "enable_speaker_diarization":True,
#     "enable_word_time_offsets":True,
#     "diarization_speaker_count":2,
}

with open(input, "rb") as f:
    audio = {"content": f.read()}
    
response = client.recognize(config, audio)

for result in response.results: 
    print('Transcript: {}'.format(result.alternatives[0].transcript))

Transcript: 我想check一check我嘅mpf account同埋有幾多錢唔該
